<a href="https://colab.research.google.com/github/vishalkandukuri97/AI-ML-Projects/blob/main/GRU_Based_Text_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
from tensorflow.keras.models import Sequential

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
from tensorflow.keras.layers import GRU, Dense, Embedding

In [8]:
import gradio as gr

In [9]:
text = """
Once upon a time in a land far away, there was a little village surrounded by lush green forests.
The villagers lived in harmony with nature, cherishing the beauty around them.
"""

In [10]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)


In [11]:
char_index = tokenizer.word_index
index_char = tokenizer.index_word
vocab_size = len(char_index) + 1


In [12]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(text)

# Flatten the list of sequences
sequences = [item for sublist in sequences for item in sublist]

# Create input-output pairs
seq_length = 40
X, y = [], []
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

In [13]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    GRU(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=64)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.0789 - loss: 3.2567
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1576 - loss: 3.2376
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1754 - loss: 3.2173
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1467 - loss: 3.1917
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1350 - loss: 3.1488
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.1467 - loss: 3.0637 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1467 - loss: 2.9458
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1601 - loss: 2.9227
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2298 - loss: 2.8995
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2178 - loss: 2.8968


In [16]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 40, 50)         │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 150)            │        90,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 26)             │         3,926 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 288,380 (1.10 MB)

 Trainable params: 96,126 (375.49 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 192,254 (751.00 KB)

In [17]:
# Step 3: Text generation function
def generate_text(seed_text, num_chars=200):
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, truncating='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_char_index = np.argmax(predicted_probs)
        predicted_char = index_char[predicted_char_index]
        seed_text += predicted_char
    return seed_text


In [18]:
def gradio_generate(seed_text, num_chars):
    return generate_text(seed_text, int(num_chars))

In [19]:
interface = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Seed Text", placeholder="Enter seed text, e.g., 'Once upon a time'"),
        gr.Slider(minimum=50, maximum=500, step=10, value=200, label="Number of Characters")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GRU-Based Text Generator",
    description="Enter a seed text and specify the number of characters to generate new text using a GRU-based neural network."
)

In [20]:
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://18744b13490adfe267.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
